# Assignment 3

**Name:** Wendy Stefany Escamilla Valadez

**e-mail:** wendy.escamilla@cucei.udg.mx

__________________________

## Minimum requirements
- **Functions** that return the different types of trajectories: Brownian Motion (**BM**),
Correlated Random Walk (**CRW**), and Lévy Flight (**LF**).
- All functions must take as arguments the **number of steps**, **speed**, and **starting**
position.
- In addition, **CRW** and **LF** functions must also take as an argument the **Cauchy coefficient**.
- Lastly, the **LF** function must also include the **Lévy exponent** (alpha) as an
argument.
_____________
- Functions that compute the metrics: Path length (**PL**), Mean Squared Displacement (**MSD**), and Turning Angle Distribution (**TAD**).
The dashboard should include at least the following functionalities:
- A panel to display the current trajectory (3D projection).
- A panel to display the graphic for the metric of choice.
______________
- Radio Items (or a similar type of selector) to select the type of trajectory (BM, CRW, LF).
- Drop down menu (or similar) to select the metric to be displayed (PL, MSD, TAD).
- Widgets to the trajectories’ parameters (integer and floating point values). These widgets
must be dynamically displayed according to the type of trajectory selected.
_____________________

# MODULES

In [173]:
import panel as pn
import panel.widgets as pnw
import plotly.graph_objects as go

import math
import pandas as pd
from scipy.stats import cauchy
from scipy.stats import levy_stable

pn.extension('plotly')

_____________________________

# Classes

In [174]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

In [176]:
def levy_flight(steps, speed, alpha, starting_x, starting_y, starting_z = 0, beta=0):
    # Generate steps size using Levy
    
    r = levy_stable.rvs(alpha, beta, loc=0, size=steps)
    initial_velocity = Vec2d(1, 0)
    initial_position = Vec2d(starting_x, starting_y)
    z_pos = starting_z


    # Start trajectory
    trajectory = [{'x_pos': starting_x, 'y_pos': starting_y, 'z_pos': starting_z}]
    position = initial_position
    velocity = initial_velocity

    # Genetare trajectory
    for i in range(steps):
        steps = r[i]  
        velocity = velocity.rotated(steps)  # Rotating the velocity vector
        length_traveled = velocity.get_length() * steps * speed  # Calculate the distance traveled

        # New position
        position = position + Vec2d(velocity.x * steps, velocity.y * steps)
        z_pos += length_traveled

        # Store new position
        trajectory.append({'x_pos': position.x, 'y_pos': position.y, 'z_pos': z_pos})

    # Change list to DataFrame
    trajectory_df = pd.DataFrame(trajectory)

    return trajectory_df

In [175]:
def correlated_random_walk(steps, speed, alpha, starting_x, starting_y, starting_z = 0, beta=0):
    # init velocity vector
    velocity = Vec2d(speed,0)
    initial_velocity = Vec2d(1, 0)
    initial_position = Vec2d(starting_x, starting_y)
    z_pos = starting_z

    trajectory = [{'x_pos': initial_position.x, 'y_pos': initial_position.y, 'z_pos': z_pos}]
    position = initial_position
    velocity = initial_velocity

    for i in range(steps):

        angle = cauchy.rvs(alpha) 

        # Rotating the velocity vector
        velocity = velocity.rotated(angle)

        length_traveled = velocity.get_length()

        # New position
        position = position + Vec2d(velocity.x, velocity.y)
        z_pos += length_traveled

        # Store new position
        trajectory.append({'x_pos': position.x, 'y_pos': position.y, 'z_pos': z_pos})
    
    #Change list to DataFrame
    trajectory_df = pd.DataFrame(trajectory)

    return trajectory_df

_____________________________

# Panel

In [177]:
# Widgets
trajectory_selector = pnw.Select(options=['BM', 'CRW', 'LF'], name=' ', width=90, value='BM')
steps_input = pnw.IntInput(start=0, end=1000000, step=1, value=1000, name='Steps:', width=80)
speed_input = pnw.FloatInput(start=0, end=1000000, step=.1, value=1000, name='Speed:', width=80)
starting_x_input = pnw.FloatInput(start=0, end=1000000, step=.1, value=5, name='Starting X:', width=80)
starting_y_input = pnw.FloatInput(start=0, end=1000000, step=.1, value=5, name='Starting Y:', width=80)
alpha_input = pnw.FloatInput(start=0.1, end=1000, step=.1, value=.7, name='Alpha', width=200)
cauchy_input = pnw.FloatInput(start=0, end=1000000, step=.1, value=1000, name='Cauchy', width=200)

metric_selector = pnw.Select(options=['PL', 'MSD', 'TAD'], value='PL', name='', width=90)


# For defect inputs disable
cauchy_input.visible = False
alpha_input.visible = False


In [178]:
# Function what connect to widgets
@pn.depends(trajectory_selector, steps_input, speed_input, starting_x_input, starting_y_input, alpha_input, cauchy_input)
def plot_trajectory(trajectory_selector, steps_input, speed_input, starting_x_input, starting_y_input, alpha_input, cauchy_input):
    if trajectory_selector == 'BM':
        trajectory = [{'x_pos': 0, 'y_pos': 4, 'z_pos': 0}]
        trajectory_df = pd.DataFrame(trajectory)
    elif trajectory_selector == 'CRW':
        trajectory_df = correlated_random_walk(steps_input,speed_input,cauchy_input,starting_x_input,starting_y_input)
    else:
        trajectory_df = levy_flight(steps_input,speed_input,alpha_input,starting_x_input,starting_y_input)

    fig_3d = go.Figure()

    fig_3d.add_trace(go.Scatter3d(
        x=trajectory_df['x_pos'], 
        y=trajectory_df['y_pos'], 
        z=trajectory_df['z_pos'], 
        marker = dict(size=2),
        line = dict(color='green',width=1),
        mode = 'lines',
        name = trajectory_selector,
        showlegend = True,
        
    ))

    fig_3d.update_yaxes(
        scaleanchor ="x",
        scaleratio = 1
    )

    fig_3d.update_layout(title=f"{trajectory_selector} Trayectory")


    return fig_3d
2

2

In [179]:
# Function what connect to widgets for plot metric
@pn.depends(trajectory_selector, steps_input, speed_input, starting_x_input, starting_y_input, alpha_input, cauchy_input, metric_selector)
def plot_metric(trajectory_selector, steps_input, speed_input, starting_x_input, starting_y_input, alpha_input, cauchy_input, metric_selector):
    if trajectory_selector == 'PL':
        trayectory = '1'
        # trayectory = random_walk(steps)
    elif trajectory_selector == 'MSD':
        trayectory= '2'
        # trayectory= correlated_random_walk(steps)
    else:
        trayectory = '3'
        # trayectory = levy_stable(steps)
    

    fig_2d = go.Figure()

    fig_2d.add_trace(go.Scatter(
        x = [1,4],
        y = [3,5],
        marker = dict(size=2),
        line = dict(color='green',width=1),
        mode = 'lines',
        name = metric_selector,
        showlegend = True,
        
    ))

    fig_2d.update_yaxes(
        scaleanchor ="x",
        scaleratio = 1
    )

    fig_2d.update_layout(title=f"Metric {metric_selector}")
    
    return fig_2d


In [180]:
# Function to show/hide cauchy coefficient input based on filter selection
def update_inputs(event):

    if trajectory_selector.value == 'LF':
        alpha_input.visible = True
        cauchy_input.visible = False
    elif trajectory_selector.value == 'CRW':
        cauchy_input.visible = True
        alpha_input.visible = False
    else:
        cauchy_input.visible = False
        alpha_input.visible = False

# funtion update to the trayectory_selector
trajectory_selector.param.watch(update_inputs, 'value')

Watcher(inst=Select(name=' ', options=['BM', 'CRW', 'LF'], value='CRW', width=90), cls=<class 'panel.widgets.select.Select'>, fn=<function update_inputs at 0x00000241A09D09A0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [181]:
# Draw panel
pn.Column(
    pn.Row(
        pn.Column(
            pn.pane.Markdown("# PANEL"),
            pn.Row(
                pn.pane.Markdown("### **Trajectory:**"),
                trajectory_selector
            ),
            pn.pane.Markdown("## Parameters"),
            pn.Column(
                pn.Row(
                    steps_input, 
                    speed_input
                )
            ), 
            pn.Column(
                pn.Row(
                    starting_x_input, 
                    starting_y_input
                )
            ),
            alpha_input,
            cauchy_input,
            pn.pane.Markdown("# Metric"),
            pn.Row(
                pn.pane.Markdown("### **Metric type:**"),
                metric_selector
            ),
        ), 
        pn.pane.Plotly(plot_trajectory, height=400, width=500),
        # pn.pane.Plotly(plot_metric, height=400, width=500)
    )
)


BokehModel(combine_events=True, render_bundle={'docs_json': {'321e9755-f448-4f79-b963-f65876a92cfa': {'version…